# Séance 2

## 1. Chargement des données

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set()

df = pd.read_excel("datasets/Notes.xlsx", sheet_name=0, header=0, index_col=0)
if df.isnull().values.any():
    print("Missing data!")
    exit(1)
df

## 2. Description univariée des variables

In [ ]:
df.boxplot()

print("Mean:")
print(df.mean())

print("\nVariance:")
print(df.var())

print("\nStandard deviation:")
print(df.std())

**Remarque:** comparaison calcul variance numpy/pandas/formule

In [ ]:
print("Pandas native:", df.MATH.var())

print("Numpy (default args) :", np.var(df.MATH.to_numpy(), axis=0))

print("Numpy (ddof = 1) :", np.var(df.MATH.to_numpy(), axis=0, ddof=1))

math_vals: np.array = df.MATH.to_numpy()
print("V(x) :", 1/(math_vals.size) * ((math_vals-math_vals.mean())**2).sum())
print("S²(x) :", 1/(math_vals.size - 1) * ((math_vals-math_vals.mean())**2).sum())

## 2. Analyse bivariée

On utilise un scatter_matrix pour observer les intercorrélations entre nos variables

In [ ]:
pd.plotting.scatter_matrix(df)

In [ ]:
print("Covariance:")
print(df.cov())

print("\nCorrélation:")
print(df.corr())

sns.heatmap(df.corr(), vmin=-1, vmax=1, cmap="coolwarm", annot=True)

Explication ACP:

$$\Sigma_x=P D P^{-1}$$

$\Sigma_x$ = Matrice de valeurs propres diagonale (= `acp.explained_variance_`)  
$D$ = Matrice de valeurs propres diagonale (= `acp.explained_variance_`)  
$P$ = Matrice de passage / matrice des vecteurs propres (= `acp.components_.transpose()`)  

In [ ]:
from sklearn.decomposition import PCA

acp = PCA()
Xproj = acp.fit_transform(df)
print(f"Valeurs projetées:\n{Xproj}\n")

Xpass = acp.components_.transpose()
print(f"Matrice de passage:\n{Xpass}\n")

print(f"Variance par vecteur propre (colonne de Xpass):\n{acp.explained_variance_}\n")
print(f"Contribution par vecteur propre:\n{acp.explained_variance_ratio_}\n")

print(f"Inertie: {acp.explained_variance_.sum()}")

labels = [f"V{i}" for i, _ in enumerate(acp.explained_variance_ratio_)]
plt.bar(labels, acp.explained_variance_ratio_, width=0.25, label='Variance ratio')
plt.plot(labels, acp.explained_variance_ratio_.cumsum(), 'r.-', label='Cumulative sum')
plt.title("Pareto diagram")
plt.legend()

On voit que les deux premiers vecteurs propres suffisent à expliquer presque l'entièreté de la variance: on représente en deux dimensions selons les axes des deux premiers vecteurs:

In [ ]:
plt.scatter(Xproj[:,0], Xproj[:,1])
for i, name in enumerate(df.index):
    plt.annotate(name, (Xproj[i,0], Xproj[i,1]))

### Interprétation des axes

On peut regarder les coordonnées dans la matrice de passage:
$$
V_1 = \begin{pmatrix}
0.5151694 \\
0.50761286 \\
0.49227894 \\
0.48434607
\end{pmatrix} ;
V_2 = \begin{pmatrix}
-0.56865175 \\
-0.3712665 \\
 0.65815337  \\
 0.32500849
\end{pmatrix}
$$

On peut en conclure que:  
- $V_1$ est liée aux 4 variables d'entrée de façon positive et ~ égale.
- $V_2$ nous indique si l'élève est plus bon en langues qu'en sciences.

Si on a des données plus complexes il vaut mieux passer par un cercle de corrélation:

In [ ]:
# Calculate correlations between new data (Xproj) and original columns (df):
corvar = np.zeros((len(df.columns), 2))
for i, col in enumerate(df.columns):
    corvar[i, 0] = np.corrcoef(Xproj[:,0], df.iloc[:, i])[0, 1]
    corvar[i, 1] = np.corrcoef(Xproj[:,1], df.iloc[:, i])[0, 1]
print(f"Correlation coefficients:\n{corvar}")

# Cercle des corrélations
fig, axes = plt.subplots(figsize=(8,8))
axes.set_xlim(-1,1)
axes.set_ylim(-1,1)

# On ajoute les axes
plt.plot([-1,1],[0,0],color='silver',linestyle='-',linewidth=1)
plt.plot([0,0],[-1,1],color='silver',linestyle='-',linewidth=1)
# On ajoute un cercle
cercle = plt.Circle((0,0),1,color='blue',fill=False)
axes.add_artist(cercle)
plt.xlabel("Composante principale 1")
plt.ylabel("Composante principale 2")
plt.title('Cercle des corrélations')
plt.scatter(corvar[:,0],corvar[:,1])
#affichage des étiquettes (noms des variables)
for j, _ in enumerate(df.columns):
  plt.annotate(df.columns[j],(corvar[j,0],corvar[j,1]))

plt.show()